In [1]:
import numpy as np
import binascii
import hashlib
import base58
import struct
import datetime
import hashlib

## Parse blockchain

In [2]:
def startsWithOpNCode(pub):
    try:
        intValue = int(pub[0:2], 16)
        if intValue >= 1 and intValue <= 75:
            return True
    except:
        pass
        return False

In [3]:
def publicKeyDecode(pub):
    if pub.lower().startswith(b'76a914'):
        pub = pub[6:-4]
        result = (b'\x00') + binascii.unhexlify(pub)
        h5 = hashlib.sha256(result)
        h6 = hashlib.sha256(h5.digest())
        result += h6.digest()[:4]
        return base58.b58encode(result)
    elif pub.lower().startswith(b'a9'):
        return ""
    elif startsWithOpNCode(pub):
        pub = pub[2:-2]
        h3 = hashlib.sha256(binascii.unhexlify(pub))
        h4 = hashlib.new('ripemd160', h3.digest())
        result = (b'\x00') + h4.digest()
        h5 = hashlib.sha256(result)
        h6 = hashlib.sha256(h5.digest())
        result += h6.digest()[:4]
        return base58.b58encode(result)
    return ""

In [4]:
def reverse(h):
    byte_array = bytearray(h)
    h_new = bytearray(b'0')
    for i in range(0,len(byte_array),2):
        h_new[i:i+1] = byte_array[len(byte_array)-1-i-1], byte_array[len(byte_array)-1-i]
        
    return bytes(h_new)

In [5]:
def hexToInt(value):
    return int(binascii.hexlify(value), 16)

def hexToStr(value):
    return binascii.hexlify(value)

def readVarInt(blockFile):
    varInt = ord(blockFile.read(1))
    returnInt = 0
    if varInt < 0xfd:
        return varInt
    if varInt == 0xfd:
          returnInt = readShortLittleEndian(blockFile)
    if varInt == 0xfe:
        returnInt = readIntLittleEndian(blockFile)
    if varInt == 0xff:
        returnInt = readLongLittleEndian(blockFile)
    return int(binascii.hexlify(returnInt), 16)

In [6]:
def stringLittleEndianToBigEndian(string):
    string = binascii.hexlify(string)
    n = len(string) / 2
    fmt = '%dh' % n
    return struct.pack(fmt, *reversed(struct.unpack(fmt, string)))

def readIntLittleEndian(blockFile):
    return struct.pack(">I", struct.unpack("<I", blockFile.read(4))[0])

def readShortLittleEndian(blockFile):
    return struct.pack(">H", struct.unpack("<H", blockFile.read(2))[0])

def readLongLittleEndian(blockFile):
    return struct.pack(">Q", struct.unpack("<Q", blockFile.read(8))[0])


In [7]:
def readInput(blockFile):
    previousHash = binascii.hexlify(blockFile.read(32)[::-1])
    outId = binascii.hexlify(blockFile.read(4))
    scriptLength = readVarInt(blockFile)
    scriptSignatureRaw = hexToStr(blockFile.read(scriptLength))
    scriptSignature = scriptSignatureRaw
    seqNo = binascii.hexlify(readIntLittleEndian(blockFile))

    print("\n" + "Input")
    print("-" * 20)
    print("> Previous Hash: ", previousHash)
    print("> Out ID: ", outId)
    print("> Script length: " + str(scriptLength))
    print("> Script Signature (PubKey) Raw: ", scriptSignatureRaw)
    print("> Script Signature (PubKey): ", scriptSignature)
    print("> Seq No: ", seqNo)

In [8]:
def readOutput(blockFile):
    value = hexToInt(readLongLittleEndian(blockFile)) / 100000000.0
    scriptLength = readVarInt(blockFile)
    scriptSignatureRaw = hexToStr(blockFile.read(scriptLength))
    scriptSignature = scriptSignatureRaw
    address = ''
    try:
        address = publicKeyDecode(scriptSignature)
    except:
        address = ''
  
    print("\n" + "Output")
    print("-" * 20)
    print("> Value: " + str(value))
    print("> Script length: " + str(scriptLength))
    print("> Script Signature (PubKey) Raw: ", scriptSignatureRaw)
    print("> Script Signature (PubKey): ", scriptSignature)
    print("> Address: ", address)

In [9]:
def readTransaction(blockFile):
    extendedFormat = False
    beginByte = blockFile.tell()
    inputIds = []
    outputIds = []
    version = hexToInt(readIntLittleEndian(blockFile)) 
    cutStart1 = blockFile.tell()
    cutEnd1 = 0
    inputCount = readVarInt(blockFile)
    
    print("\n\n" + "Transaction")
    print("-" * 100)
    print("Version: " + str(version))
    
    print("\nInput Count: " + str(inputCount))
    for inputIndex in range(0, inputCount):
        inputIds.append(readInput(blockFile))
    outputCount = readVarInt(blockFile)
    print("\nOutput Count: " + str(outputCount))
    for outputIndex in range(0, outputCount):
        outputIds.append(readOutput(blockFile))
        
    lockTime = hexToInt(readIntLittleEndian(blockFile))
    if lockTime < 500000000:
        print("\nLock Time is Block Height: " + str(lockTime))
    else:
        print("\nLock Time is Timestamp: " + datetime.datetime.fromtimestamp(lockTime).strftime('%d.%m.%Y %H:%M'))


In [10]:
def process_block(blockFile):
    magicNumber = binascii.hexlify(blockFile.read(4))
    blockSize = hexToInt(readIntLittleEndian(blockFile))
    version = hexToInt(readIntLittleEndian(blockFile))
    previousHash = reverse(binascii.hexlify(blockFile.read(32)))
    merkleHash = reverse(binascii.hexlify(blockFile.read(32)))
    creationTimeTimestamp = hexToInt(readIntLittleEndian(blockFile))
    creationTime = datetime.datetime.fromtimestamp(creationTimeTimestamp).strftime('%d.%m.%Y %H:%M')
    bits = hexToInt(readIntLittleEndian(blockFile))
    nonce = hexToInt(readIntLittleEndian(blockFile))
    countOfTransactions = readVarInt(blockFile)
    
    print("Magic Number: ", magicNumber)
    print("Blocksize: " + str(blockSize))
    print("Version: " + str(version))
    print("Previous Hash: ", previousHash)
    print("Merkle Hash: ", merkleHash)
    print("Time: " , creationTime)
    print("Bits: " + str(bits))
    print("Nonce: " + str(nonce))
    print("Count of Transactions: " + str(countOfTransactions))
    
    for transactionIndex in range(0, countOfTransactions):
        readTransaction(blockFile)

In [11]:
i = 0
with open('blocks/blk00002.dat', 'rb') as file:
    while i < 10:
        print(i)
        process_block(file)
        print("")
        print("+"*100)
        print("")
        i+=1

cd62087541e19a29c71a488ac'
> Address:  b'12kC829aVXYMZAeU74CzRTyBJMefbyE36F'

Output
--------------------
> Value: 0.41658093
> Script length: 25
> Script Signature (PubKey) Raw:  b'76a91456296f8135ecfd607e5f120e581524c9e18c6e8b88ac'
> Script Signature (PubKey):  b'76a91456296f8135ecfd607e5f120e581524c9e18c6e8b88ac'
> Address:  b'18rakVUcsP27WAbEKK8AGbjYwZ1L4TbpBv'

Lock Time is Block Height: 0


Transaction
----------------------------------------------------------------------------------------------------
Version: 1

Input Count: 2

Input
--------------------
> Previous Hash:  b'6524a0942fcddbe8e40110c0db23a8dccc6a64372c2aea6ca311c7065200ffe6'
> Out ID:  b'00000000'
> Script length: 140
> Script Signature (PubKey) Raw:  b'493046022100ad9287df9dd5b5acc77622ced4bc16a31ba06bdc0a427eaf0a30371e213f15a7022100949708bc75618a3047a5f340bfec196e4e8ba7af390af4904cb74a762ad4000401410469ab556322ae827177305c29c3b68e22a646b57423a74c02c5a658323feae54525e1641723d959f76c27442a4d00d89beac3a22e0dd3f1361a

## VarInts (base128)

In [12]:
def encodeVarInt(h):
    n = int.from_bytes(binascii.unhexlify(h), byteorder='little')
    l = 0
    tmp = []
    data = ""

    while True:
        tmp.append(n & 0x7F)
        if l != 0:
            tmp[l] |= 0x80
        if n <= 0x7F:
            break
        n = (n >> 7) - 1
        l += 1

    tmp.reverse()
    for i in tmp:
        data += format(i, '02x')
    return data

def decodeVarInt(data):
    n = 0
    i = 0
    while True:
        d = int(data[2 * i:2 * i + 2], 16)
        n = n << 7 | d & 0x7F
        if d & 0x80:
            n += 1
            i += 1
        else:
            return n


def parseVarInt(value, offset=0):

    data = value[offset:offset+2]
    offset += 2
    more_bytes = int(data, 16) & 0x80
    while more_bytes:
        data += value[offset:offset+2]
        more_bytes = int(value[offset:offset+2], 16) & 0x80
        offset += 2

    return data, offset

In [13]:
int_t = b'00100000'

In [14]:
varint_t = encodeVarInt(int_t)

In [15]:
decodeVarInt(varint_t)

4096

## Hash160 to Address

In [16]:
hash160 = 'f7b11ed316ddb4db47ae5da0fce0a171f290d4c4'

In [17]:
key_hash = '00' + hash160

In [18]:
sha = hashlib.sha256()
sha.update( bytearray.fromhex(key_hash) )
checksum = sha.digest()
sha = hashlib.sha256()
sha.update(checksum)
checksum = sha.hexdigest()[0:8]

In [19]:
address = base58.b58encode(bytes(bytearray.fromhex(key_hash + checksum))).decode('utf-8')

In [20]:
address

'1Pag5vaogpwnHEouACXGQwCcnogAhrekzd'